# MÉTODO DE LOS ELEMENTOS FINITOS EN ELEMENTOS BARRA

Elaborado por: Efrain Rodriguez Infantes


La aplicación de este ejemplo se hará a una armadura hiperestática con la siguiente configuración:

<img src="Ejemplo2.png">

$E = 2\cdot 10^{11} N/m^2$,
$A = 0.005 m^2$

Fuente de imagen: Dr. Seán Carroll

In [172]:
#----------------- CARGA DE LAS LIBRERÍAS A USAR ---------------#
import numpy as np
import plotly.graph_objects as go
import math


## 1. PRE-PROCESAMIENTO

### 1.1. INGRESO DE DATOS

In [173]:
#----------------- PROPIEDADES DE LOS MATERIALES ---------------#

E = 2*10**11 #N/m2
A = 0.005 #m2

#----------------- DEFINICIÓN DE LOS NUDOS ---------------#


nodos = np.array([[0,0],
                  [9.804,18],
                  [20,18],
                  [30.195,18],
                  [40,18],
                  [50.715,18],
                  [62.939,18],
                  [76.204,18],
                  [90,18],
                  [103.79,18],
                  [117.06,18],
                  [129.28,18],
                  [140,18],
                  [149.8,18],
                  [160,18],
                  [170.19,18],
                  [180,0],
                  [170.16,5.194],
                  [160,6.948],
                  [149.8,5.194],
                  [140,0],
                  [129.28,4.939],
                  [117.06,8.61],
                  [103.79,10.87],
                  [90,11.633],
                  [76.204,10.87],
                  [62.939,8.61],
                  [50.715,4.939],
                  [40,0],
                  [30.195,5.194],
                  [20,6.948],
                  [9.804,5.194]])
            
    
#----------------- DEFINICIÓN DE LAS BARRAS ---------------#


barras = np.array([
                   #Barras del perímetro
                   [1,2],
                   [2,3],
                   [3,4],
                   [4,5],
                   [5,6],
                   [6,7],
                   [7,8],
                   [8,9],
                   [9,10],
                   [10,11],
                   [11,12],
                   [12,13],
                   [13,14],
                   [14,15],
                   [15,16],
                   [16,17],
                   [17,18],
                   [18,19],
                   [19,20],
                   [20,21],
                   [21,22],
                   [22,23],
                   [23,24],
                   [24,25],
                   [25,26],
                   [26,27],
                   [27,28],
                   [28,29],
                   [29,30],
                   [30,31],
                   [31,32],
                   [32,1],
                   #Barras verticales
                   [2,32],
                   [3,31],
                   [4,30],
                   [5,29],
                   [6,28],
                   [7,27],
                   [8,26],
                   [9,25],
                   [10,24],
                   [11,23],
                   [12,22],
                   [13,21],
                   [14,20],
                   [15,19],
                   [16,18],
                   #Barras diagonales
                   [3,32],
                   [3,30],
                   [4,29],
                   [6,29],
                   [7,28],
                   [8,27],
                   [9,26],
                   [9,24],
                   [10,23],
                   [11,22],
                   [12,21],
                   [14,21],
                   [15,20],
                   [15,18],
                   ])

#----------------- CONDICIONES DE CONTORNO (APOYOS) ---------------#


restricciones = [1,2,33,34,41,42,57,58]


#----------------- CARGAS ---------------#

gdl=2*len(nodos) # Número de grados de libertad (2 por nudo)

fuerzas = np.array([np.zeros(gdl)]).T #Creación del vector de fuerzas, un vector columna.
                                        #Si bien es cierto, las fuerzas en los apoyos serán diferentes de cero
                                        #a estas alturas no significará un gran cambio en los resultados, pues posteriormente
                                        #se calcularán las fuerzas de los apoyos

# Ingreso de fuerzas (las matrices en python se empiezan a contar desde 0),
# por lo que si se quiere ingresar una fuerza en el grado de libertad 4,
# en el vector de fuerzas se hará en el índice 3, que es la cuarta posición del vector


fuerzas[11] = -200000
fuerzas[13] = -200000
fuerzas[15] = -200000
fuerzas[17] = -200000
fuerzas[19] = -200000
fuerzas[21] = -200000
fuerzas[23] = -200000




### 1.2. PLOTEO DE ESTRUCTURA PARA CONFIRMAR EL CORRECTO INGRESO DE DATOS

In [174]:
fig = go.Figure()

#Ploteo de barras
contador = 1
for barra in barras:
    nodo_i = barra[0]   #Nodo i de la barra
    nodo_j = barra[1]   #Nodo j de la barra

    i_x = nodos[nodo_i-1, 0] #Coordenada X del nudo i de la barra
    i_y = nodos[nodo_i-1, 1] #Coordenada Y del nudo i de la barra
    j_x = nodos[nodo_j-1, 0] #Coordenada X del nudo j de la barra
    j_y = nodos[nodo_j-1, 1] #Coordenada Y del nudo j de la barra

    fig.add_trace(go.Scatter(x=[i_x,j_x], y=[i_y,j_y], 
                            mode='lines+markers',
                            name='barra ' + str(contador),
                            #showlegend=False,
                            #line = dict(color='black'),
                            ))
    contador += 1

#Ploteo de fuerzas



for i in range(len(fuerzas)):
    fuerza = fuerzas[i][0]

    if fuerza != 0:
        x_fuerza = nodos[math.trunc(i/2), 0]
        y_fuerza = nodos[math.trunc(i/2), 1]

        if i%2 == 0:
            ax_fuerza = fuerza
            ay_fuerza = 0
        else:
            ax_fuerza = 0
            ay_fuerza = fuerza

        fig.add_annotation(
            x=x_fuerza,
            y=y_fuerza,
            xref="x",
            yref="y",
            text=str(fuerza/1000)+" kN",
            showarrow=True,
            font=dict(
                family="Courier New, monospace",
                size=12,
                color="#000000"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#000000",
            ax=-ax_fuerza*.0003,
            ay=ay_fuerza*.0003,
            opacity=0.7
            )
            
fig.update_layout(title='GRÁFICA DE ARMADURA A CALCULAR',
                   xaxis_title='Coordenadas X (m)',
                   yaxis_title='Coordenadas Y (m)')

fig.show()


## 2. PROCESAMIENTO

### 2.1 Determinar la rigidez local de cada elemento

\begin{equation*}
[Ke]=
\begin{bmatrix}
AE/L & 0 & -AE/L & 0\\
0 & 0 & 0 & 0\\
-AE/L & 0 & AE/L & 0\\
0 & 0 & 0 & 0
\end{bmatrix}
\end{equation*}


### 2.2 Matriz de transformación [T]

\begin{equation*}
[T]=
\begin{bmatrix}
\cos{\theta} & -\sin{\theta} & 0 & 0\\
\sin{\theta} & \cos{\theta} & 0 & 0\\
0 & 0 & \cos{\theta} & -\sin{\theta} \\
0 & 0 & \sin{\theta} & \cos{\theta}
\end{bmatrix}
\end{equation*}


<img src="elemento_orientacion.png">
Fuente: Kennane, 2013

### 2.3 Matriz de rigidez global por cada elemento reducido

\begin{equation*}
[K^{elem}_{glob.red}]=[T]^T \cdot [Ke] \cdot [T]
\end{equation*}

### 2.4 Matriz de rigidez global del elemento

En este paso se ensambla la matriz de rigidez global por cada elemento reducido en su correspondiente fila y columna en la matriz de rigidez global de cada elemento según el GDL, para posteriormente sumar el aporte de cada elemento a la matriz de rigidez global de la estructura.

In [175]:
def calculosPorBarra(barraNo):
    
    nodo_i = barras[barraNo][0]
    nodo_j = barras[barraNo][1]

    xi = nodos[nodo_i-1][0]
    yi = nodos[nodo_i-1][1]
    xj = nodos[nodo_j-1][0]
    yj = nodos[nodo_j-1][1]

    coordX=[xi,xj]
    coordY=[yi,yj]

    dx = xj-xi #distancia en la componente X
    dy = yj-yi #distancia en la componente Y

    puntero =np.array([[nodo_i*2-1, nodo_i*2, nodo_j*2-1, nodo_j*2]])    #Vector que contiene los GDL del nodo inicial y final

    L = math.sqrt(dx**2+dy**2) #Longitud total de la barra

    # 2.1 Matriz de rigidez local
    Ke=(E*A/L)*np.array([   [1,0,-1,0],
                            [0,0,0,0],
                            [-1,0,1,0],
                            [0,0,0,0]])


    # 2.2 Matriz de transformación

    # Se necesita determinar el ángulo de la barra con respecto a la horizontal, para ello se definen los siguientes casos:

    if(dx>0 and dy==0): #Vector horizontal apuntando a la derecha (theta = 0°)
        theta = 0
    elif(dx==0 and dy>0): #Vector vertical apuntando hacia arriba (theta = 90°)
        theta = math.pi/2
    elif(dx<0 and dy==0): #Vector horizontal apuntando a la izquierda (theta = 180°)
        theta = math.pi
    elif(dx==0 and dy<0): #Vector vertical apuntando hacia abajo (theta = 270°)
        theta = 3*math.pi/2
    elif(dx>0 and dy>0): # Primer cuadrante (0°<theta<90°)
        theta = math.atan(dy/dx)
    elif(dx<0 and dy>0): # Segundo cuadrante (90°<theta<180°)
        theta = math.atan(dy/dx)+math.pi
    elif(dx<0 and dy<0): # Tercer cuadrante (180°<theta<270°)
        theta = math.atan(dy/dx)+math.pi
    elif(dx>0 and dy<0): # Cuarto cuadrante (270°<theta<360°)
        theta = math.atan(dy/dx)+2*math.pi


    S_theta = math.sin(theta) #Seno del ángulo theta
    C_theta = math.cos(theta) #Coseno del ángulo C_theta

    T = np.array([  [C_theta,-S_theta,0,0],
                    [S_theta,C_theta,0,0],
                    [0,0,C_theta,-S_theta],
                    [0,0,S_theta,C_theta]])

    # 2.3 Matriz de rigidez global reducida del elemento

    Ke_gr = np.dot(np.dot(np.transpose(T),Ke),T)

    # 2.4 Matriz de rigidez global del elemento

    Ke_g = np.zeros((gdl,gdl)) # Creación de la matriz con solo ceros

    # Se reemplazará los ceros con los valores de Ke_gr en su respectiva posición según su GDL
    for i in range(4):
        for j in range(4):
            
            a = puntero[0,i]-1 #GDL para primera posición
            b = puntero[0,j]-1 #GDL para segunda posición            
                
            Ke_g[a,b] = Ke_gr[i,j] #Asignación del valor correspondiente a su fila y columna según el GDL correspondiente


    
    return L,coordX,coordY,puntero, Ke, T, Ke_gr, Ke_g    
       

In [176]:
#Creación de la lista que contendrá los cálculos de cada barra
CalcBarra = []

#Cálculo por cada elemento 
for i in range(len(barras)):
    L,coordX,coordY,puntero, Ke, T, Ke_gr, Ke_g = calculosPorBarra(i)
    calculos = {    'Barra' : i+1,
                    'Longitud': L,
                    'Coordenadas X':coordX,
                    'Coordenadas Y':coordY,
                    'Puntero':puntero,
                    'Rigidez Elemento':Ke,
                    'Matriz Transpuesta':T,
                    'Rigidez Elemento Glob. Red.':Ke_gr,
                    'Rigidez Elemento Global':Ke_g
                }
    CalcBarra.append(calculos)
    


### 2.5 Ensamblaje de la matriz global de la estructura

Para hallar la matriz global de la estructura simplemente se tiene que sumar las matrices globales de cada elemento, los cuales lo tenemos guardado en la lista "CalcBarra".

In [177]:
K=np.zeros((gdl,gdl)) #Se crea la matriz de rigidez de la estructura con ceros que se llenarán posteriormente

for i in range(len(barras)):
    K = K + CalcBarra[i]['Rigidez Elemento Global']

### 2.6 Reducción de la matriz global de la estructura y la matriz de fuerzas
Se reduce la matriz global de la estructura en donde los desplazamientos son conocidos. Es decir, para este caso se eliminaran las filas de los GDL en donde los desplazamientos son ceros (los apoyos). Además, se eliminan las filas también de los GDL en donde los desplazamientos son conocidos, esto con la finalidad de resolver una ecuación matricial en donde las únicas incógnitas son los desplazamientos.

In [178]:
# Reducción de la matriz global de la estructura
K_red=K #Asignamos la matriz global a la matriz global reducida para poder eliminar filas y columnas

K_red=np.delete(K_red,[elemento - 1 for elemento in restricciones],axis=0) #Elimina la fila
K_red=np.delete(K_red,[elemento - 1 for elemento in restricciones],axis=1) #Elimina la columna

#Reducción del vector de fuerzas
F_red = fuerzas

F_red=np.delete(F_red,[elemento - 1 for elemento in restricciones],axis=0) #Se eliminan las filas en donde los desplazamientos son conocidos



### 2.7 Solución de la ecuación $[K]_{red}[\delta]_{red}=[F]_{red}$

En este paso se hallarán los desplazamientos desconocidos con el sistema reducido y luego se completará con ceros en los GDL donde ya se conocen los desplazamientos (apoyos)

In [179]:
delta_desc=np.dot(np.linalg.inv(K_red),F_red) #Se resuelve la ecuación matricial

#Se crea una lista complementaria a la de restricciones, que muestre las posiciones en donde no se conocen los desplazamientos

restric_complem =[]

for i in range(gdl):
    restric_complem.append(i+1)

for restriccion in restricciones:
    restric_complem.remove(restriccion)

#Se completa el vector de desplazamientos global de la estructura

delta = np.array([np.zeros(gdl)]).T #Vector de ceros para reemplazar en su lugar los desplazamientos hallados

contador=0
for i in restric_complem:
    delta[i-1]=delta_desc[contador]
    contador += 1

print(delta)


[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-4.15354210e-03]
 [-7.31296861e-04]
 [-4.47344854e-03]
 [ 3.24548026e-04]
 [-6.53370719e-03]
 [ 1.79476448e-03]
 [-8.86968012e-03]
 [-1.62933381e-18]
 [-1.14224542e-02]
 [-1.53420791e-02]
 [-1.21437049e-02]
 [-4.36348297e-02]
 [-8.77990323e-03]
 [-7.60224620e-02]
 [ 1.41053415e-06]
 [-8.56987327e-02]
 [ 8.78075975e-03]
 [-7.60371653e-02]
 [ 1.21455524e-02]
 [-4.36376016e-02]
 [ 1.14265232e-02]
 [-1.53574718e-02]
 [ 8.87190329e-03]
 [ 1.62974219e-18]
 [ 6.53652317e-03]
 [ 1.79505317e-03]
 [ 4.47445302e-03]
 [ 3.23803661e-04]
 [ 4.15173512e-03]
 [-7.24241370e-04]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.32545447e-03]
 [ 1.01179001e-05]
 [ 1.38008034e-03]
 [ 2.87059887e-04]
 [ 1.59602302e-03]
 [ 9.44987822e-04]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-4.69512568e-03]
 [-1.66810520e-02]
 [-3.91015777e-03]
 [-4.48949639e-02]
 [-8.71662100e-04]
 [-7.65439174e-02]
 [ 2.18670051e-06]
 [-8.57173575e-02]
 [ 8.76788921e-04]
 [-7.65291585e-02]
 [ 3.9140843

## 3. POST-PROCESAMIENTO

### 3.1 Cálculo de las fuerzas desconocidas globales

Se resuelve nuevamente la ecuación que se presenta a continuación, en donde la incógnita será el vector de fuerzas

\begin{equation*}
[K][\delta]=[F]
\end{equation*}

In [180]:
F = np.dot(K,delta)
print(F)

[[ 1.25006477e+05]
 [-1.07209364e+05]
 [-2.32830644e-10]
 [ 2.91038305e-11]
 [ 0.00000000e+00]
 [ 3.63797881e-12]
 [ 0.00000000e+00]
 [-2.91038305e-11]
 [ 0.00000000e+00]
 [-3.87740912e-26]
 [ 2.32830644e-10]
 [-2.00000000e+05]
 [ 1.16415322e-10]
 [-2.00000000e+05]
 [-3.49245965e-10]
 [-2.00000000e+05]
 [-1.60071068e-10]
 [-2.00000000e+05]
 [ 2.32830644e-10]
 [-2.00000000e+05]
 [-3.49245965e-10]
 [-2.00000000e+05]
 [ 1.16415322e-09]
 [-2.00000000e+05]
 [-2.32830644e-10]
 [ 1.29246971e-26]
 [ 2.32830644e-10]
 [ 0.00000000e+00]
 [-2.32830644e-10]
 [-3.63797881e-12]
 [ 5.82076609e-11]
 [ 0.00000000e+00]
 [-1.25075811e+05]
 [-1.07236842e+05]
 [-5.82076609e-11]
 [ 2.91038305e-11]
 [ 2.91038305e-11]
 [ 1.40971679e-11]
 [-2.91038305e-11]
 [-5.82076609e-11]
 [ 9.31494287e+05]
 [ 8.07223833e+05]
 [-4.65661287e-10]
 [ 0.00000000e+00]
 [-4.65661287e-10]
 [-1.16415322e-09]
 [ 9.31322575e-10]
 [-1.39698386e-09]
 [-1.43359102e-10]
 [ 6.60656951e-09]
 [-4.65661287e-10]
 [-1.16415322e-09]
 [-1.3969838

### 3.2 Cálculo de las fuerzas internas y desplazamientos locales de cada barra

* Desplazamientos locales
\begin{equation*}
\begin{bmatrix}
u'_{xi}\\
u'_{yi}\\
u'_{xj}\\
u'_{yj}
\end{bmatrix}
= [T][u]
\end{equation*}

* Elongación de barra
\begin{equation*}
elongación=u'_{xj}-u'_{xi}
\end{equation*}

* Fuerza interna
\begin{equation*}
P=\frac{EA}{L} \cdot elongación
\end{equation*}


In [181]:
def desplBarraGlob(barraNo):
    nodo_i = barras[barraNo][0]
    nodo_j = barras[barraNo][1]

    puntero =np.array([[nodo_i*2-1, nodo_i*2, nodo_j*2-1, nodo_j*2]]) #Vector que contiene los GDL del nodo inicial y final

    # Desplazamientos globales de los nudos de cada barra

    delta_barra_global=np.zeros((4,1))

    for i in range(4):
        delta_barra_global[i,0] = delta[puntero[0,i]-1,0]

    # Desplazamientos 
    return delta_barra_global

In [182]:
for i in range(len(barras)):
    delta_barra_global = desplBarraGlob(i)

    #Se irá agregando a la lista los resultados de cada barra
    CalcBarra[i]['Delta Barra Global']=delta_barra_global
    CalcBarra[i]['Delta Barra Local']=np.dot(CalcBarra[i]['Matriz Transpuesta'],CalcBarra[i]['Delta Barra Global'])
    CalcBarra[i]['Elongación']=CalcBarra[i]['Delta Barra Local'][2,0]-CalcBarra[i]['Delta Barra Local'][0,0]
    CalcBarra[i]['Fuerza interna']=E*A/CalcBarra[i]['Longitud']*CalcBarra[i]['Elongación']
    
    if round(CalcBarra[i]['Fuerza interna'],3)>0:
        CalcBarra[i]['Estado']='Tracción'
    elif round(CalcBarra[i]['Fuerza interna'],3)<0:
        CalcBarra[i]['Estado']='Compresión'
    elif round(CalcBarra[i]['Fuerza interna'],0)==0:
        CalcBarra[i]['Estado']='No esforzada'
        


### 3.3 Ploteo de resultados

In [183]:
#Se crea un data frame con el que se plotearán los gráficos

# df=pd.DataFrame({'barra':

# })


###### PLOTEO DE COMPRESIÓN O TRACCIÓN

fig = go.Figure()

for i in range(len(barras)):

    if CalcBarra[i]['Estado']=='Compresión':
        barraColor = 'blue'
    elif CalcBarra[i]['Estado']=='Tracción':
        barraColor = 'red'
    elif CalcBarra[i]['Estado']== 'No esforzada':
        barraColor = 'green'
    fig.add_trace(go.Scatter(x=CalcBarra[i]['Coordenadas X'], y=CalcBarra[i]['Coordenadas Y'],
                             mode='lines+markers',
                             name=CalcBarra[i]['Estado'],
                             line = dict(color=barraColor)

    ))

fig.update_layout(title='Resultados de Armadura',
                   xaxis_title='Coordenadas X (m)',
                   yaxis_title='Coordenadas Y (m)')

fig.show()


###### PLOTEO DE DEFORMACIONES

factor = 50 #Es el factor de exageración con el que se quiere deformar la estructura

fig = go.Figure()

for i in range(len(barras)):

    if CalcBarra[i]['Estado']=='Compresión':
        barraColor = 'blue'
    elif CalcBarra[i]['Estado']=='Tracción':
        barraColor = 'red'
    elif CalcBarra[i]['Estado']== 'No esforzada':
        barraColor = 'green'
    
    fig.add_trace(go.Scatter(x=CalcBarra[i]['Coordenadas X'], y=CalcBarra[i]['Coordenadas Y'],
                             mode='lines+markers',
                             name=CalcBarra[i]['Estado'],
                             line = dict(color='grey'),
                             showlegend = False
                             ))

    fig.add_trace(go.Scatter(x=[CalcBarra[i]['Coordenadas X'][0]+delta[CalcBarra[i]['Puntero'][0,0]-1,0]*factor,CalcBarra[i]['Coordenadas X'][1]+delta[CalcBarra[i]['Puntero'][0,2]-1,0]*factor], 
                             y=[CalcBarra[i]['Coordenadas Y'][0]+delta[CalcBarra[i]['Puntero'][0,1]-1,0]*factor,CalcBarra[i]['Coordenadas Y'][1]+delta[CalcBarra[i]['Puntero'][0,3]-1,0]*factor],
                             mode='lines+markers',
                             #name=CalcBarra[i]['Estado'],
                             line = dict(color='red'),
                             showlegend=False

    ))

fig.update_layout(title='Deformación de la Armadura',
                   xaxis_title='Coordenadas X (m)',
                   yaxis_title='Coordenadas Y (m)')

fig.show()

#PLOTEO DE BARRAS CON FUERZAS INTERNAS





### 3.3 Resultados de fuerzas internas

In [184]:
print("Fuerzas internas en barras")
for i in range(len(barras)):
    print("Barra "+str(i+1)+":")
    print(str(round(CalcBarra[i]['Fuerza interna']/1000,2)) + " kN")


Fuerzas internas en barras
Barra 1:
-65.6 kN
Barra 2:
-31.38 kN
Barra 3:
-202.09 kN
Barra 4:
-238.24 kN
Barra 5:
-238.24 kN
Barra 6:
-59.0 kN
Barra 7:
253.58 kN
Barra 8:
636.51 kN
Barra 9:
636.65 kN
Barra 10:
253.56 kN
Barra 11:
-58.84 kN
Barra 12:
-238.3 kN
Barra 13:
-238.3 kN
Barra 14:
-202.16 kN
Barra 15:
-31.67 kN
Barra 16:
-65.9 kN
Barra 17:
-105.77 kN
Barra 18:
-9.79 kN
Barra 19:
-9.79 kN
Barra 20:
87.25 kN
Barra 21:
952.68 kN
Barra 22:
577.26 kN
Barra 23:
172.22 kN
Barra 24:
-26.48 kN
Barra 25:
-26.48 kN
Barra 26:
172.36 kN
Barra 27:
577.22 kN
Barra 28:
952.93 kN
Barra 29:
87.23 kN
Barra 30:
-9.73 kN
Barra 31:
-9.73 kN
Barra 32:
-105.96 kN
Barra 33:
57.61 kN
Barra 34:
-3.3 kN
Barra 35:
-66.38 kN
Barra 36:
0.0 kN
Barra 37:
-101.1 kN
Barra 38:
-133.99 kN
Barra 39:
-71.07 kN
Barra 40:
-2.93 kN
Barra 41:
-71.07 kN
Barra 42:
-133.9 kN
Barra 43:
-101.34 kN
Barra 44:
-0.0 kN
Barra 45:
-66.38 kN
Barra 46:
-3.32 kN
Barra 47:
57.86 kN
Barra 48:
-134.93 kN
Barra 49:
139.14 kN
Barra 50:
75.